# <p align="center"><b style="color:#1f6feb;"> **🤖✨Comparative Neural Machine Translation: LSTM vs Transformer for English–French Translation🤖✨**</p>


### ‎‧₊**˚✿[My Name]✿˚** : **[Mohamed Reda Ramadan Khamis]**
### ‎‧₊**˚✿[My Phone Number]✿˚** : **[01554725661]**

# **LSTM TRANSLATOR**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rajpulapakura/english-to-french-small-dataset")

print("Path to dataset files:", path)

100%|██████████| 4.51M/4.51M [00:00<00:00, 6.47MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/rajpulapakura/english-to-french-small-dataset/versions/3


In [ ]:
import pandas as pd
df = pd.read_csv(path + '/english_french.csv')
df.head()

,English,French
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [ ]:
df.tail()

,English,French
229798,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
229799,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
229800,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
229801,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...
229802,I went drinking with one of my boyfriend's fri...,« Je suis allée boire avec un ami de mon compa...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229803 entries, 0 to 229802
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  229803 non-null  object
 1   French   229803 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Add special tokens to target (French) sentences
df["French"] = df["French"].apply(lambda x: "<start> " + x + " <end>")
df = df.sample(n=150000, random_state=42).reset_index(drop=True)
input_texts = df["English"]
target_texts = ["<start> " + text + " <end>" for text in df["French"]]

# Tokenization
src_tokenizer = Tokenizer(filters='')
tgt_tokenizer = Tokenizer(filters='')

src_tokenizer.fit_on_texts(input_texts)
tgt_tokenizer.fit_on_texts(target_texts)

src_sequences = src_tokenizer.texts_to_sequences(input_texts)
tgt_sequences = tgt_tokenizer.texts_to_sequences(target_texts)

max_encoder_seq_length = max(len(seq) for seq in src_sequences)
max_decoder_seq_length = max(len(seq) for seq in tgt_sequences)

# Padding
encoder_input_data = pad_sequences(src_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences([seq[:-1] for seq in tgt_sequences], maxlen=max_decoder_seq_length-1, padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in tgt_sequences], maxlen=max_decoder_seq_length-1, padding='post')

In [ ]:
df.head()

,English,French
0,"If you give me a book, I'll read it.","<start> Si vous me donnez un livre, je le lira..."
1,"We'll leave tomorrow, weather permitting.","<start> Nous partirons demain, si le temps le ..."
2,"I'm so happy, I feel like I could fly.",<start> Je suis si heureux que je m'élève dans...
3,Pull over here.,<start> Arrêtez-vous ici ! <end>
4,Moses came down from the mountain bearing divi...,<start> Moïse descendit de la montagne en port...


In [ ]:
max_encoder_seq_length

55

In [ ]:
max_decoder_seq_length

60

In [ ]:
encoder_input_data.shape

(150000, 55)

In [ ]:
decoder_input_data.shape

(150000, 59)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Hyperparameters
embedding_dim = 64
lstm_units = 60

# Vocabulary sizes
num_encoder_tokens = len(src_tokenizer.word_index) + 1
num_decoder_tokens = len(tgt_tokenizer.word_index) + 1

# ----- Encoder -----
encoder_inputs = Input(shape=(None,))
encoder_embedding_layer = Embedding(num_encoder_tokens, embedding_dim)
encoder_embedding = encoder_embedding_layer(encoder_inputs)
_, state_h, state_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# ----- Decoder -----
decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(num_decoder_tokens, embedding_dim)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm_layer = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm_layer(decoder_embedding, initial_state=encoder_states)
decoder_dense_layer = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense_layer(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 64)  │  1,634,368 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 64)  │  2,843,520 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 60),      │     30,000 │ embedding[0][0]   │
│                     │ (None, 60),       │            │                   │
│                     │ (None, 60)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     30,000 │ embedding_1[0][0… │
│                     │ 60), (None, 60),  │            │ lstm[0][1],       │
│                     │ (None, 60)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  2,710,230 │ lstm_1[0][0]      │
│                     │ 44430)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,248,118 (27.65 MB)

 Trainable params: 7,248,118 (27.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_encoder_tokens

25537

In [ ]:
num_decoder_tokens

44430

In [ ]:
44430 * 64

In [ ]:
decoder_target_data.shape

(150000, 59)

In [ ]:
decoder_target_data = decoder_target_data.reshape(
    decoder_target_data.shape[0],
    decoder_target_data.shape[1],
    1
)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=3,
    validation_split=0.2
)


Epoch 1/3
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 209s 54ms/step - loss: 1.5239 - val_loss: 0.6551
Epoch 2/3
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 205s 55ms/step - loss: 0.6204 - val_loss: 0.5602
Epoch 3/3
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 205s 55ms/step - loss: 0.5240 - val_loss: 0.5090


In [ ]:
# --- Encoder inference ---
encoder_model = Model(encoder_inputs, encoder_states)

# --- Decoder inference ---
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_inf = Input(shape=(None,))
decoder_emb_inf = decoder_embedding_layer(decoder_inputs_inf)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_layer(
    decoder_emb_inf, initial_state=decoder_states_inputs)
decoder_outputs_inf = decoder_dense_layer(decoder_outputs_inf)

decoder_model = Model(
    [decoder_inputs_inf] + decoder_states_inputs,
    [decoder_outputs_inf] + [state_h_inf, state_c_inf]
)


In [ ]:
import numpy as np
def decode_sequence(input_seq):
    # Get encoder states
    states_value = encoder_model.predict(input_seq)

    # Start token
    target_seq = np.array([[tgt_tokenizer.word_index['<start>']]])
    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = next((word for word, index in tgt_tokenizer.word_index.items()
                             if index == sampled_token_index), "")

        if sampled_word == "<end>" or len(decoded_sentence.split()) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_word

        # Update
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return decoded_sentence.strip().replace("<start> ", "")

def translate(text):
    seq = src_tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_encoder_seq_length, padding='post')
    return decode_sequence(padded)


In [ ]:
print(translate("Go."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
le chien est en train de faire ça.


In [ ]:
print(translate("Go."))
print(translate("Where is the bathroom?"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
le chien est en train de faire ça.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
êtes-vous en train de la nuit ?


# **Transformer Translator**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rajpulapakura/english-to-french-small-dataset")

print("Path to dataset files:", path)

100%|██████████| 4.51M/4.51M [00:01<00:00, 3.13MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rajpulapakura/english-to-french-small-dataset/versions/3


In [ ]:
import pandas as pd
df = pd.read_csv(path + '/english_french.csv')
df.head()

,English,French
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

df = df.sample(n=150000, random_state=42).reset_index(drop=True)
df["French"] = df["French"].apply(lambda x: "<start> " + x.strip() + " <end>")

input_texts = df["English"].tolist()
target_texts = df["French"].tolist()

src_tokenizer = Tokenizer(filters='', lower=True, oov_token="<OOV>")
src_tokenizer.fit_on_texts(input_texts)
src_sequences = src_tokenizer.texts_to_sequences(input_texts)

tgt_tokenizer = Tokenizer(filters='', lower=True, oov_token="<OOV>")
tgt_tokenizer.fit_on_texts(target_texts)
tgt_sequences = tgt_tokenizer.texts_to_sequences(target_texts)

max_len_src = max(len(seq) for seq in src_sequences)
max_len_tgt = max(len(seq) for seq in tgt_sequences)

encoder_input = pad_sequences(src_sequences, maxlen=max_len_src, padding='post')
decoder_input = pad_sequences([seq[:-1] for seq in tgt_sequences], maxlen=max_len_tgt - 1, padding='post')
decoder_target = pad_sequences([seq[1:] for seq in tgt_sequences], maxlen=max_len_tgt - 1, padding='post')

vocab_size_src = len(src_tokenizer.word_index) + 1
vocab_size_tgt = len(tgt_tokenizer.word_index) + 1

decoder_target = decoder_target.reshape(decoder_target.shape[0], decoder_target.shape[1], 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention, Add
from tensorflow.keras.models import Model

# Positional Encoding Layer (simple sinusoidal)
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

# Transformer Block
def transformer_block(x, context, head_size, num_heads, ff_dim, dropout=0.1):
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, context)
    attn = Dropout(dropout)(attn)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([x, attn]))

    ffn = Dense(ff_dim, activation='relu')(out1)
    ffn = Dense(x.shape[-1])(ffn)
    ffn = Dropout(dropout)(ffn)
    return LayerNormalization(epsilon=1e-6)(Add()([out1, ffn]))

# Build the full model
def build_transformer(vocab_size_src, vocab_size_tgt, max_len_src, max_len_tgt,
                      d_model=128, ff_dim=512, num_heads=4):
    enc_inputs = Input(shape=(None,))
    dec_inputs = Input(shape=(None,))

    src_emb = Embedding(vocab_size_src, d_model)(enc_inputs)
    src_emb = PositionalEncoding(max_len_src, d_model)(src_emb)

    tgt_emb = Embedding(vocab_size_tgt, d_model)(dec_inputs)
    tgt_emb = PositionalEncoding(max_len_tgt, d_model)(tgt_emb)

    enc_out = transformer_block(src_emb, src_emb, d_model, num_heads, ff_dim)
    dec_out = transformer_block(tgt_emb, enc_out, d_model, num_heads, ff_dim)

    output = Dense(vocab_size_tgt, activation='softmax')(dec_out)

    model = Model(inputs=[enc_inputs, dec_inputs], outputs=output)
    return model


In [ ]:
# Build the model
model = build_transformer(
    vocab_size_src=vocab_size_src,
    vocab_size_tgt=vocab_size_tgt,
    max_len_src=max_len_src,
    max_len_tgt=max_len_tgt,
    d_model=128,
    ff_dim=512,
    num_heads=4
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Reshape decoder_target to match output shape (batch, seq_len, 1)
decoder_target = decoder_target.reshape(decoder_target.shape[0], decoder_target.shape[1], 1)

# Train



In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 128) │  3,268,864 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encoding │ (None, None, 128) │          0 │ embedding[0][0]   │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 128) │    263,808 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ positional_encod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, None, 128) │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, None, 128) │          0 │ positional_encod… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, None, 128) │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 512) │     66,048 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 128) │     65,664 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, None, 128) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │  5,687,168 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, None, 128) │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, None, 128) │          0 │ embedding_1[0][0] │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 128) │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 128) │    263,808 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, None, 128) │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, None, 128) │          0 │ positional_encod… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 15,479,695 (59.05 MB)

 Trainable params: 15,479,695 (59.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=3,
    validation_split=0.1
)

Epoch 1/3
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 228s 51ms/step - loss: 1.0335 - val_loss: 0.5846
Epoch 2/3
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 205s 49ms/step - loss: 0.5395 - val_loss: 0.4996
Epoch 3/3
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 205s 49ms/step - loss: 0.4457 - val_loss: 0.4460


In [ ]:
def translate(input_sentence):
    # Encode source sentence
    seq = src_tokenizer.texts_to_sequences([input_sentence])
    enc_input = pad_sequences(seq, maxlen=max_len_src, padding='post')

    # Start with <start> token
    start_token = tgt_tokenizer.word_index['<start>']
    end_token = tgt_tokenizer.word_index['<end>']
    output_seq = [start_token]

    for _ in range(max_len_tgt):
        dec_input = pad_sequences([output_seq], maxlen=max_len_tgt - 1, padding='post')
        pred = model.predict([enc_input, dec_input], verbose=0)
        next_token = np.argmax(pred[0, len(output_seq)-1])

        if next_token == end_token:
            break

        output_seq.append(next_token)

    # Convert tokens back to text
    reverse_tgt_word_index = {i: w for w, i in tgt_tokenizer.word_index.items()}
    translated_words = [reverse_tgt_word_index.get(i, '') for i in output_seq[1:]]
    return ' '.join(translated_words).strip()


In [ ]:
print(translate("How are you?"))
print(translate("Hi."))

où es-tu ?
sensass !
